In [2]:
import math
import matplotlib.pyplot as plt
import numpy as np
import random 

In [3]:
initial_board = np.zeros(shape=(5,5), dtype=np.int32)
print(initial_board)

[[0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]]


In [4]:
class State:
    def __init__(self):
        #self.father = None
        self.board = np.zeros(shape=(5,5), dtype=np.int32)
        self.player= None
        
class Player:
    def __init__(self):
        self.value = 0
        #self.moves = 0
        self.name = None
        self.pieces = []
        
        
def result(state):
    #Horizontales
    for i in range(5):
        if np.all(state.board[i][0:4] == 1) or np.all(state.board[i][1:5] == 1):
            return 1
        elif np.all(state.board[i][0:4] == -1) or np.all(state.board[i][1:5] == -1):
            return -1
        
    #Verticales
    for i in range(5):
        column = state.board[:,i]
        if np.sum(column == 1) == 4:
            if np.all(column[0:4] == 1) or np.all(column[1:5] == 1):
                return 1
        elif np.sum(column == -1) == 4:
            if np.all(column[0:4] == -1) or np.all(column[1:5] == -1):
                return -1
            
    #Diagonales       
    for i in range(-1,2):
            diag = state.board.diagonal(i)
            oppdiag = np.fliplr(state.board).diagonal(i)

            if np.sum(diag == 1) == 4 or np.sum(oppdiag == 1) == 4:
                if np.all(diag[0:4] == 1) or np.all(oppdiag[0:4] == 1):
                    return 1
                if len(diag)>4:
                    if np.all(diag[1:5] == 1) or np.all(oppdiag[1:5] == 1):
                        return 1
            elif np.sum(diag == -1) == 4 or np.sum(oppdiag == -1) == 4:
                if np.all(diag[0:4] == -1) or np.all(oppdiag[0:4] == -1):
                    return -1
                if len(diag)>4:
                    if np.all(diag[1:5] == -1) or np.all(oppdiag[1:5] == -1):
                        return -1
                    
    #Cuadrado               
    for i in range(4):
                for j in range(4):
                    c = [state.board[i][j:j+2],state.board[i+1][j:j+2]]
                    cube = np.array(c)

                    if np.all(cube == 1):
                        return 1
                    elif np.all(cube == -1):
                        return -1

    return 0
        
def question_place(player):
    print(f'{player.name} Ingresa la posicion xy:')
    pos = input()
    x_pos, y_pos = transform_values(pos)
    return x_pos, y_pos


def question_move_place(player):
    print(f'{player.name} Ingresa la posicion de la ficha a mover xy:')
    pos = input()
    x_pos, y_pos = transform_values(pos)
    print(f'{player.name} Ingresa la posicion a la cual se movera xy:')
    pos = input()
    new_x_pos, new_y_pos = transform_values(pos)
    return x_pos, y_pos, new_x_pos, new_y_pos

def validate_place(x_pos, y_pos, board):
    if(x_pos>=0 and x_pos<5 and y_pos>=0 and y_pos<5):
        if(board[x_pos][y_pos]==0):
            return True
    return False

#def validate_new_place(x_pos, y_pos, board):
    #if(x_pos>=0 and x_pos<5 and y_pos>=0 and y_pos<5):
       # if(board[x_pos][y_pos]==0):
           # return True
   # return False
        
def init_Game(player1,player2):
    start_state = State()
    board = start_state.board
    
    for i in range(4):
        #Player1
        x_pos,y_pos=question_place(player1)
        while(validate_place(x_pos,y_pos, board)==False):
            print("lugar invalido.. vuelve a intentar")
            x_pos,y_pos=question_place(player1)
        board[x_pos][y_pos]=player1.value
        print(board)
        res=result(state)
        if(res!=0):
            winner(player1,player2,res)
            break
        #Player2
        x_pos,y_pos=question_place(player2)
        while(validate_place(x_pos,y_pos, board)==False):
            print("lugar invalido.. vuelve a intentar")
            x_pos,y_pos=question_place(player2)
        board[x_pos][y_pos]=player2.value
        print(board)
        res=result(state)
        if(res!=0):
            winner(player1,player2,res)
            break
        
    return board
        
        
def move(x_pos, y_pos, new_x_pos, new_y_pos, player, state):
    if (x_pos==new_x_pos and y_pos==new_y_pos):
        print("Posiciones iguales, vuelva a intentar..")
        return False
    if (state.board[x_pos][y_pos]==player.value):
        if(state.board[new_x_pos][new_y_pos]==0):
            adjacentPlaces = possible_moves(x_pos, y_pos, state.board)
            if [new_x_pos, new_y_pos] in adjacentPlaces:
                state.board[new_x_pos][new_y_pos]=player.value
                state.board[x_pos][y_pos]=0
                return True
            else:
                print("Destino no adyacente, vuelva a intentar..")
                return False
        else:
            print("Destino ocupado, vuelva a intentar..")
            return False
    else:
        print("ficha seleccionada vacia o del jugador oponente, vuelva a intentar..")
        return False

def possible_moves(x_pos, y_pos, board):
    validMoves = []
    AllMoves = [[0, -1], [0, +1], [-1, 0] ,[+1, 0], [-1, +1], [+1, -1], [-1, -1], [+1, +1]]

    for i in AllMoves:
        x = x_pos + i[0]
        y = y_pos + i[1]

        if( 0 <= x <= 4 and 0 <= y <= 4):
            if(validate_place(x,y, board)==True):
                validMoves.append([x,y]) 
    return validMoves

        
def game(player1,player2,movelimits):
    state = State()
    state.board = init_Game(player1,player2)
    print("Fase 2 Mover Fichas")
    for i in range(movelimits):
        #player 1
        state.player=player1
        x_pos, y_pos, new_x_pos, new_y_pos=question_move_place(player1)
        while(move(x_pos, y_pos, new_x_pos, new_y_pos, player1, state)==False):
            x_pos, y_pos, new_x_pos, new_y_pos=question_move_place(player1)
        print(state.board)
        res=result(state)
        if(res!=0):
            winner(player1,player2,res)
            break
        
        #player 2
        state.player=player2
        x_pos, y_pos, new_x_pos, new_y_pos=question_move_place(player2)
        while(move(x_pos, y_pos, new_x_pos, new_y_pos, player2, state)==False):
            x_pos, y_pos, new_x_pos, new_y_pos=question_move_place(player2)
        print(state.board)
        res=result(state)
        res=result(state)
        if(res!=0):
            winner(player1,player2,res)
            break
    
def transform_values(pos):
    rows = {"A":0,"B":1,"C":2,"D":3,"E":4}
    columns = {"1":0,"2":1,"3":2,"4":3,"5":4}  
    x_pos=rows[pos[0]]
    y_pos=columns[pos[1]]
    return x_pos, y_pos

def winner(player1,player2,res):
    if(res==player1.value):
        print(f' Gano: {player1.name}')
    else:
        print(f' Gano: {player2.name}')
    

In [5]:
p1=Player()
p1.value=1
p1.name="Max"
p2=Player()
p2.value=-1
p2.name="Min"
movelimits=3

game(p1,p2,movelimits)

Max Ingresa la posicion xy:


KeyboardInterrupt: Interrupted by user

In [98]:
class Action:
    def __init__(self):
        self.piece=None
        self.placeMove=None

def get_actions(state):
    actions = []
    player=state.player
    pieces=player.pieces
    for piece in pieces:
        places = possible_moves(piece[0], piece[1],state.board)
        for place in places:
            action=Action()
            action.piece=piece
            action.placeMove=place
            actions.append(action)        
    return actions
    
    
def print_actions(actions):
    for action in actions:
        x_pos, y_pos=return_values(action.piece)
        a_pos, b_pos=return_values(action.placeMove)
        print(f'{x_pos}{y_pos} mover a {a_pos}{b_pos}')
        
        
def return_values(pos):
    rows = {0:"A",1:"B",2:"C",3:"D",4:"E",10:"0"}
    columns = {0:1,1:2,2:3,3:4,4:5, 10:"0"}  
    x_pos=rows[pos[0]]
    y_pos=columns[pos[1]]
    return x_pos, y_pos

In [27]:
p1=Player()
p1.value=1
p1.name="Max"
p1.pieces=[[0, 0], [0, 1], [1,0] ,[1, 2]]

state=State()
state.board=[[ 1,  1,  0,  0,  0],
             [ 1,  0,  1,  0,  0],
             [-1, -1, -1,  0,  0],
             [ 0,  0, -1,  0,  0],
             [ 0,  0,  0,  0,  0]]
state.player=p1

actions=get_actions(state)
print_actions(actions)


A1 mover a B2
A2 mover a A3
A2 mover a B2
B1 mover a B2
B3 mover a B2
B3 mover a B4
B3 mover a A3
B3 mover a A4
B3 mover a C4


In [99]:
def get_initial_actions(state):
    actions=[]
    board=state.board
    for x in range(5):
        for y in range(5):
            if board[x][y]==0:
                action=Action()
                action.piece=[10,10]
                action.placeMove=[x,y]
                actions.append(action)  
    return actions


In [37]:
state=State()
state.board=[[ 1,  1,  0,  0,  0],
             [ 1,  0,  1,  0,  0],
             [-1, -1, -1,  0,  0],
             [ 0,  0, -1,  0,  0],
             [ 0,  0,  0,  0,  0]]

actions=get_initial_actions(state)
print(actions)

[[0, 2], [0, 3], [0, 4], [1, 1], [1, 3], [1, 4], [2, 3], [2, 4], [3, 0], [3, 1], [3, 3], [3, 4], [4, 0], [4, 1], [4, 2], [4, 3], [4, 4]]


In [100]:
def get_total_actions(state):
    value=state.player.value
    board=state.board
    count=len(np.where(board==value)[0])
    print(count)
    if(count==4):
        return get_actions(state)
    else:
        return get_initial_actions(state)
    

In [101]:
state=State()
state.board[0][0]=1
state.board[1][0]=1
state.board[0][1]=1
state.board[0][3]=-1
p1=Player()
p1.value=1
p1.pieces=[[0, 0], [0, 1], [1,0]]
state.player=p1
actions=get_total_actions(state)
print_actions(actions)

3
00 mover a A3
00 mover a A5
00 mover a B2
00 mover a B3
00 mover a B4
00 mover a B5
00 mover a C1
00 mover a C2
00 mover a C3
00 mover a C4
00 mover a C5
00 mover a D1
00 mover a D2
00 mover a D3
00 mover a D4
00 mover a D5
00 mover a E1
00 mover a E2
00 mover a E3
00 mover a E4
00 mover a E5


In [103]:
state=State()
state.board[0][0]=1
state.board[1][0]=1
state.board[0][1]=1
state.board[1][2]=1
state.board[0][3]=-1
p1=Player()
p1.value=1
p1.pieces=[[0, 0], [0, 1], [1,0], [1,2]]
state.player=p1
actions=get_total_actions(state)
print_actions(actions)

4
A1 mover a B2
A2 mover a A3
A2 mover a B2
B1 mover a B2
B1 mover a C1
B1 mover a C2
B3 mover a B2
B3 mover a B4
B3 mover a A3
B3 mover a C3
B3 mover a C2
B3 mover a C4


In [102]:
def result_init(state, action):
    if(action.piece==[10,10]):
        state.board[action.placeMove[0]][action.placeMove[1]]=state.player.value
    else:
        state.board[action.piece[0]][action.piece[1]]=0
        state.board[action.placeMove[0]][action.placeMove[1]]=state.player.value
        
        
    #Horizontales
    for i in range(5):
        if np.all(state.board[i][0:4] == state.player.value) or np.all(state.board[i][1:5] == state.player.value):
            return state.player.value
        
    #Verticales
    for i in range(5):
        column = state.board[:,i]
        if np.sum(column == state.player.value) == 4:
            if np.all(column[0:4] == 1) or np.all(column[1:5] == state.player.value):
                return state.player.value
            
    #Diagonales       
    for i in range(-1,2):
            diag = state.board.diagonal(i)
            oppdiag = np.fliplr(state.board).diagonal(i)

            if np.sum(diag == state.player.value) == 4 or np.sum(oppdiag == state.player.value) == 4:
                if np.all(diag[0:4] == state.player.value) or np.all(oppdiag[0:4] == state.player.value):
                    return state.player.value
                if len(diag)>4:
                    if np.all(diag[1:5] == state.player.value) or np.all(oppdiag[1:5] == state.player.value):
                        return state.player.value
                    
    #Cuadrado               
    for i in range(4):
                for j in range(4):
                    c = [state.board[i][j:j+2],state.board[i+1][j:j+2]]
                    cube = np.array(c)

                    if np.all(cube == state.player.value):
                        return state.player.value

    return 0

In [105]:
state=State()
state.board[0][0]=1
state.board[1][0]=1
state.board[0][1]=1
state.board[1][2]=1
state.board[0][3]=-1
p1=Player()
p1.value=1
p1.pieces=[[0, 0], [0, 1], [1,0], [1,2]]
state.player=p1

action=Action()
action.piece=[1,2]
action.placeMove=[1,1]

actions=get_total_actions(state)
print_actions(actions)
res=result_init(state,action)
print("res ",res)
print(state.board)

4
A1 mover a B2
A2 mover a A3
A2 mover a B2
B1 mover a B2
B1 mover a C1
B1 mover a C2
B3 mover a B2
B3 mover a B4
B3 mover a A3
B3 mover a C3
B3 mover a C2
B3 mover a C4
res  1
[[ 1  1  0 -1  0]
 [ 1  1  0  0  0]
 [ 0  0  0  0  0]
 [ 0  0  0  0  0]
 [ 0  0  0  0  0]]


In [107]:
state=State()
state.board[0][0]=1
state.board[1][0]=1
state.board[0][1]=1
state.board[0][3]=-1
p1=Player()
p1.value=1
p1.pieces=[[0, 0], [0, 1], [1,0]]
state.player=p1

action=Action()
action.piece=[10,10]
action.placeMove=[1,1]

actions=get_total_actions(state)
print_actions(actions)
res=result_init(state,action)
print("res ",res)
print(state.board)

3
00 mover a A3
00 mover a A5
00 mover a B2
00 mover a B3
00 mover a B4
00 mover a B5
00 mover a C1
00 mover a C2
00 mover a C3
00 mover a C4
00 mover a C5
00 mover a D1
00 mover a D2
00 mover a D3
00 mover a D4
00 mover a D5
00 mover a E1
00 mover a E2
00 mover a E3
00 mover a E4
00 mover a E5
res  1
[[ 1  1  0 -1  0]
 [ 1  1  0  0  0]
 [ 0  0  0  0  0]
 [ 0  0  0  0  0]
 [ 0  0  0  0  0]]


In [9]:
#Heuristica
from itertools import groupby

def count_positive_negative_in_row(row):
    count_positive = 0
    count_negative = 0
    for i in row:
        if not (i == 0):
            if i > 0:
                count_positive += 1
            else:
                count_negative +=1
    return [count_positive ,count_negative]

assert count_positive_negative_in_row([1, 1, 1,-1,-1]) == [3, 2]
assert count_positive_negative_in_row([0, 0, 0, 1, 1]) == [2, 0]
assert count_positive_negative_in_row([-1, 0, -1, 0, 1]) == [1, 2]
assert count_positive_negative_in_row([1, 1, 0, 1, 1]) == [4, 0]

def is_consecutive(x, y):
    return (x == y) and not (x == 0)

def splitInArrays(row):
    return [list(g) for k,g in groupby(row,lambda x:x==0) if not k]

def count_consecutive_in_row(row):
    count_positive = 0
    count_negative = 0
    listPositives = [ x if x == 1 else 0 for x in row ] # convierte los -1 en 0, me quedo solo con 1s
    listNegatives = [ 1 if x == -1 else 0 for x in row ] # convierte los 1 en 0 y despues los -1 en 1s, me quedo solo con 1s

    listPositivesArrays= splitInArrays(listPositives) #separa por 0s, [1,1,0,1,1] => [[1,1][1,1]]
    listNegativesArrays= splitInArrays(listNegatives) #separa por 0s, [1,1,0,1,1] => [[1,1][1,1]]

    respPositivos = []
    for arr in listPositivesArrays:
        respPositivos.append(sum(arr))
    respPositivos =  0 if respPositivos == [] else math.prod(respPositivos)  

    respNegativos = []
    for arr in listNegativesArrays:
        respNegativos.append(sum(arr))
    respNegativos =  0 if respNegativos == [] else math.prod(respNegativos)  
    return [respPositivos, respNegativos ]

def eval_row(row):
    #cantPos * cantPosConsect - cantNeg*cantNegConsect
    cantPos, cantNeg = count_positive_negative_in_row(row)
    cantPosConsect, cantNegConsect = count_consecutive_in_row(row)
    print(cantPos, "*", cantPosConsect, "-" ,cantNeg, "*",cantNegConsect)
    return cantPos * cantPosConsect - cantNeg*cantNegConsect

assert eval_row([1, 1, 0, 1, 1]) == 16
assert eval_row([1, 1, 0, -1, 1]) == 5
assert eval_row([1, 1, 0, -1, -1]) == 0
assert eval_row([1, 0, 1, 0, 1]) == 3 #3(1)(1)(1) - 0
assert eval_row([-1, 0, -1, 0, -1]) == -3
assert eval_row([-1, 0, 0, 0, -1]) == -2
assert eval_row([1, 0, 0, 0, 1]) == 2 #2(1)

def eval_rows(board):
    res=0
    for i in range(5):
        res=res+eval_row(board[i,:])
        
    return res


row = np.zeros(shape=5, dtype=np.int32)
row[0]=1
row[1]=1
row[3]=1
row[4]=1
print(row)
eval_row(row)

4 * 4 - 0 * 0
3 * 2 - 1 * 1
2 * 2 - 2 * 2
3 * 1 - 0 * 0
0 * 0 - 3 * 1
0 * 0 - 2 * 1
2 * 1 - 0 * 0
[1 1 0 1 1]
4 * 4 - 0 * 0


16

In [71]:
board = np.zeros(shape=(5,5), dtype=np.int32)
board[1][2]=1
board[0][2]=1
board[0][0]=1
board[0][3]=-1
print(board)


print(eval_rows(board))

[[ 1  0  1 -1  0]
 [ 0  0  1  0  0]
 [ 0  0  0  0  0]
 [ 0  0  0  0  0]
 [ 0  0  0  0  0]]
2


In [72]:
#Player Max
pmax=Player()
pmax.value=1
pmax.name="Max"

#Player Min
pmin=Player()
pmin.value=1
pmin.name="Min"

max_depth = 2
def cut_off(state, depth):
    return depth == max_depth


def min_max_with_depth(state):
    values=[]
    player=state.player
    actions=get_initial_actions(state)
    #Player max
    if(state.player.value == 1):
        for action in actions:
            v=min_value(result(copy.copy(state),action),-math.inf, math.inf, 0)
            values.append(v)
            index=np.argmax(values)
    #Player min       
    if(state.player.value == -1):
        for action in actions:
            v=max_value(result(copy.copy(state),action),-math.inf, math.inf, 0)
            values.append(v)
            index=np.argmin(values)
            
        
    return actions[index]

def EVAL(state):
    res_eval=0
#    player=state.player
#     if player.value == 1:
#         res_eval = len(np.where(state.board == -1)[0])
#     else:
#         res_eval = len(np.where(state.board == 1)[0])*-1
        
        
    res_eval=eval_rows(state.board)
    #res_eval=eval_colum(colum)
    #...
    return res_eval

def min_value(state,alpha, betha, depth):
    state.player=pmin
    if cut_off(state, depth):
        return EVAL(state)
    v = math.inf
    actions=get_initial_actions(state)
    for action in actions:
        v = min(v, max_value(result(copy.deepcopy(state), action), alpha, beta, depth+1))
        #prunning
        if v<=alpha:
            return v
        beta = min(beta, v)
        
    return v


def max_value(state,alpha, betha, depth):
    state.player=pmax
    if cut_off(state, depth):
        return EVAL(state)
    v = -math.inf
    actions=get_initial_actions(state)
    for action in actions:
        v = max(v, min_value(result(copy.deepcopy(state), action), alpha, beta, depth+1))
        #prunning
        if v>=beta:
            return v
        beta = max(alpha, v)
        
    return v

In [73]:
p1=Player()
p1.value=1
p1.name="Max"
#p1.pieces=[[0, 0], [0, 1], [1,0] ,[1, 2]]

state=State()
state.board[1][2]=1
state.board[0][2]=1
state.board[0][0]=1
state.board[0][3]=-1
state.player=p1

print(state.board)
print(EVAL(state))


[[ 1  0  1 -1  0]
 [ 0  0  1  0  0]
 [ 0  0  0  0  0]
 [ 0  0  0  0  0]
 [ 0  0  0  0  0]]
2
